#Learning Sklearn Pipeline



references: [Sklearn Pipeline Playground](https://www.kaggle.com/chmaxx/)

## House Price Regression

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')


from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv(
    '/content/drive/My Drive/Data/House_price/train.csv',
    encoding = 'ISO-8859-1'
)

df_test = pd.read_csv(
    '/content/drive/My Drive/Data/House_price/test.csv',
    encoding = 'ISO-8859-1'
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Let's se the data

In [2]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Pipilene magic's starts here! :P

**Sklearn documentation about Pipeline:**


---


1.   *Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.*

2.   *The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.*



**Pipeline allows to chain transformers with a estimator :)**

This is a simple pipeline for example:


In [3]:
#drop categorical features
X = df.select_dtypes("number").drop("SalePrice", axis=1)
y = df.SalePrice

#Setup a pipeline with an Imputer(fills in missing values), a Scaler and a 
#LinearRegression classifier
pipe = make_pipeline(SimpleImputer(), RobustScaler(), LinearRegression())

#crossvalidation and print
print(f"R2 score: {cross_val_score(pipe, X, y)}")

R2 score: [0.84258561 0.79289509 0.69765464]


**Working with categorical features**

In [0]:
#name of columns with numerical features
num_cols = df.drop("SalePrice", axis=1).select_dtypes("number").columns
#name of columns with categorical features
cat_cols = df.select_dtypes("object").columns

First create a pipeline for numerical

In [0]:
#Pipeline for numerical values
numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer()),
        ('scaler', RobustScaler())])
#if i want to see the result numeric_transformer.fit_transform(X)

Create a pipeline for categorical

In [0]:
#Pipeline for categorical data
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Combine

In [7]:
#ColumnTransformer combines the two created pipelines
preprocessor = ColumnTransformer(
        transformers=[
            #List of (name, transformer, column(s))
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)])

pipe = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LinearRegression())])

X = df.drop("SalePrice", axis=1)
y = df.SalePrice

print("MAGIC WAS DONE!\n")
print(45*'-'+'\n')
print(f"R2 score is: {cross_val_score(pipe, X, y)}")

MAGIC WAS DONE!

---------------------------------------------

R2 score is: [0.86801211 0.83038401 0.72109799]


### Let's show more pipeline magic using the classifiers

 Set a list with 10 common classifiers

In [0]:
classifiers = [
               DummyRegressor(),
               LinearRegression(n_jobs=-1), 
               Ridge(alpha=0.003, max_iter=30), 
               Lasso(alpha=.0005), 
               ElasticNet(alpha=0.0005, l1_ratio=.9),
               KernelRidge(alpha=0.6, kernel="polynomial", degree=2, coef0=2.5),
               SGDRegressor(),
               SVR(kernel="linear"),
               LinearSVR(),
               RandomForestRegressor(n_jobs=-1, n_estimators=350, 
                                     max_depth=12, random_state=1),
               GradientBoostingRegressor(n_estimators=500, max_depth=2),
               lgb.LGBMRegressor(n_jobs=-1, max_depth=2, n_estimators=1000, 
                                 learning_rate=0.05),
               xgb.XGBRegressor(objective="reg:squarederror", n_jobs=-1, 
                                max_depth=2, n_estimators=1500, learning_rate=0.075),
]

clf_names = [
            "dummy", 
            "linear", 
            "ridge",
            "lasso",
            "elastic",
            "kernlrdg",
            "sgdreg",
            "svr",
            "linearsvr",
            "randomforest", 
            "gbm", 
            "lgbm", 
            "xgboost"
]

**Encapsulate all our feature cleaning and engineering**

In [0]:
def clean_data(data, is_train_data=True):
    # cleaning and feature engineering here
    # here we create a new feature 
    data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

    # code that you only want to apply to your training data and not the test set
    # removing outliers from the training data
    
    if is_train_data == True:
        data = data[data.GrLivArea < 4000]
        
    return data

**Prepare our data for the pipeline**


---
Now we will use a function to separate the categorical and numerical cols


In [0]:
def prepare_data(df, is_train_data=True):
    
    # numerical or catecorical?
    numerical   = df.select_dtypes("number").copy()
    categorical = df.select_dtypes("object").copy()
    
    # get the sales price and drop the id if is train
    if is_train_data == True :
        SalePrice = numerical.SalePrice
        y = np.log1p(SalePrice)
        numerical.drop(["Id", "SalePrice"], axis=1, inplace=True)
        
    # just drop the id if is test
    else:
        numerical.drop(["Id"], axis=1, inplace=True)
        y = None
    
    # concatenate numerical and categorical 
    X = pd.concat([numerical, categorical], axis=1)
    
    # we need to return the columns because wi will use it in Column transformer
    return X, y, numerical.columns, categorical.columns

**Create the pipeline**

---
Let's encapsulate our pipeline above
36/5000
vamos encapsular nosso pipeline acima


In [0]:
#Same steps into a function

def get_pipeline(classifier, num_cols, cat_cols):
  
  numeric_transformer = Pipeline(steps=[
        ('imputer', make_pipeline(SimpleImputer(strategy='mean'))),
        ('scaler', StandardScaler())])
    
   
  categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
  preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)])
    
    
  return Pipeline(steps=[('preprocessor', preprocessor), ('classifier', classifier)])

**Don't forget crossvalidation**

In [0]:
def score_models(df):
    
    X, y, num_cols, cat_cols = prepare_data(df)

    scoring_metric = "neg_mean_squared_error"
    scores = []
    
    #for each classifier
    for clf_name, classifier in zip(clf_names, classifiers):
        
        # crate a pipeline
        clf = get_pipeline(classifier, num_cols, cat_cols)

        kfold = KFold(n_splits=3, shuffle=True, random_state=1)  
       
        results = np.sqrt(-cross_val_score(clf, X, y, cv=kfold, scoring=scoring_metric))
        scores.append([clf_name, results.mean()])

    scores = pd.DataFrame(scores, columns=["classifier", "rmse"]).sort_values("rmse", ascending=False)
   
    return scores.reset_index(drop=True)

**Now we will train \o/**

In [0]:
def train_models(df): 
    X, y, num_cols, cat_cols = prepare_data(df)
    pipelines = []
    
    for clf_name, classifier in zip(clf_names, classifiers):
        clf = get_pipeline(classifier, num_cols, cat_cols)
        clf.fit(X, y)
        pipelines.append(clf)
    
    return pipelines

**If we train we predict! kkkkkkk**

In [0]:
def predict_from_models(df_test, pipelines):
    X_test, _ , _, _ = prepare_data(df_test, is_train_data=False)
    predictions = []
    
    for pipeline in pipelines:
        preds = pipeline.predict(X_test)

        predictions.append(np.expm1(preds))
    
    return predictions

### THE TIME OF MY LIFE (Finally we will use)

In [15]:
df = clean_data(df)
df_test = clean_data(df_test, is_train_data=False)

scores_cv = score_models(df)
print(scores_cv)

models = train_models(df)
predictions = predict_from_models(df_test, models)

prediction_final = pd.DataFrame(predictions[2:]).mean().T.values

submission = pd.DataFrame({'Id': df_test.Id.values, 'SalePrice': prediction_final})
submission.to_csv(f"submission.csv", index=False)

,classifier,rmse
0,dummy,0.395788
1,sgdreg,0.245645
2,randomforest,0.138232
3,linearsvr,0.137680
4,linear,0.130697
5,svr,0.129446
6,xgboost,0.120122
7,lgbm,0.120102
8,gbm,0.119977
9,ridge,0.117331
